# Prediction of overlapping spans with spaCy's SpanCategorizer

**Motivation**:

Annotations in GGPONC are often overlapping or nested.

For instance, `Versagen einer Behandlung mit Oxaliplatin und Irinotecan`
- is a *Finding*
- which contains a *Therapeutic Procedure*: `Behandlung mit Oxaliplatin und Irinotecan`:
    - which in turn contains two *Clinical Drug* names: (`Oxaliplatin` and `Irinotecan`).

Standard IOB-encoded labels, and most NER implementations, can only model one label per token, so by default we consider the longest surrounding mention span only in the IOB-based / HuggingFace implementation (in this case, the *Finding*).

**Solution**:

Instead of token-level labels, we use spaCy's new [SpanCategorizer](https://spacy.io/api/spancategorizer/) implementation to predict overlapping mention spans as a SpanGroup in a spaCy document.

## Training

See the `spacy` folder in the root directory of the project. The model configuration can be found at `configs` and training can be run through a spaCy project (see `spacy/run_training.sh`). 

*Note:* We have currently not optimized the many hyperparameters related to span suggestion and model training. However, performance is close to the HuggingFace models evaluated on non-nested mention spans.

## Inference

In [ ]:
import sys
sys.path.append('../spacy')

In [ ]:
import spacy
from spacy.tokens import Doc, Span
import snomed_spans #TODO: import needed to enable custom spaCy components, is there another way?

In [ ]:
nlp = spacy.load('../data/models/spacy')

In [ ]:
doc = nlp("""Versagen einer Behandlung mit Oxaliplatin und Irinotecan""")

### Grascco Samples

In [ ]:
doc = nlp("""6.04.2029: Nachdebridement am Kopf, VAG-Wechsel linke Hand""")


In [ ]:
doc = nlp("6.04.2029: Nachdebridement am Kopf, VAG-Wechsel linke Hand""Röntgen : Rippstein I : Gute Hüftkopfepiphysenkonturgebung , minimale Lateralisation , li. etwas stärker als re. , noch übergreifende Pfannendächer , Shenton-Menard-Linie nicht wesentlich unterbrochen , Pfannendachwinkel Ii. 30° , re. ebenfalls knapp 30° .""")


In [ ]:
from pathlib import Path
import json
from sklearn.metrics import f1_score
from spacy.training import docs_to_json
from sklearn.metrics import precision_recall_fscore_support
from spacy.tokens import DocBin
from spacy.training import Corpus
from spacy.training import Example
from spacy.scorer import Scorer
from spacy.vocab import Vocab






folder_raw = "/Users/leon.sarodnik/Documents/GitHub/ggponc_annotation/grassco_project_2023-01-05_0021/source"
manual_annotated_file = "/Users/leon.sarodnik/Documents/GitHub/ggponc_annotation/grassco_project_2023-01-05_0021/grassco_converted/spacy/test.spacy"
p = Path(r'/Users/leon.sarodnik/Documents/GitHub/ggponc_annotation/grassco_project_2023-01-05_0021/source').glob('*.txt')
files = [x for x in p if x.is_file()]



print("loading .spacy file ...")
#gold_annotation = nlp.from_disk(manual_annotated_file)
#doc_bin = DocBin().from_disk(manual_annotated_file)




print("working ...")
text = ""
#text = ""
for file in files:
    text += file.read_text(encoding="utf-8")
docs = nlp(text)

In [ ]:

doc_bin = DocBin().from_disk(manual_annotated_file)


gold_docs  = list(doc_bin.get_docs(nlp.vocab))

#scorer = Scorer()
#scores = scorer.score(examples)

#gold_annotation = spacy.Corpus.v1(manual_annotated_file, gold_preproc=True)
docs = []
for file in files:
    docs.append(file.read_text(encoding="utf-8"))
print("Files merged...")


# Loop over the gold standard data
#for gold_doc in gold_docs:
#    ents1 = [(gold_doc.text, gold_doc.label_) for ent in gold_doc.ents]
    # Process the text with the model
# Compare the model's predicted annotations with the gold standard
#for doc in docs:
#    ents2 = [(doc.text, doc.label_) for ent in doc.ents]

print("NLP Pipe...")
docs_all = nlp.pipe(gold_docs, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"],batch_size = 10)


print(gold_docs[1])
print("Building Examples...")
examples = []
for  i, doc in enumerate(docs_all):
    examples.append(Example((doc), gold_docs[i]))
    


scorer = Scorer(nlp)

print("eval...")
scores = scorer.score(examples)


#example_object = iter(docs)


#examples = Example(Doc.from_docs(docs), Doc.from_docs(gold_docs))

scorer.score()


#print("Entities F-Score:", scorer.scores["ents_f"])
#print("Entities Precision:", scorer.scores["ents_p"])
#print("Entities Recall:", scorer.scores["ents_r"])


In [ ]:
scorer = Scorer()

scores = scorer.score(examples)

print("Entities F-Score:", scorer.scores["ents_f"])
print("Entities Precision:", scorer.scores["ents_p"])
print("Entities Recall:", scorer.scores["ents_r"])


In [ ]:
sentence['offsets']

In [ ]:
import json

def find_annotated_entities(sentence, document):
    s_offsets = sentence['offsets']
    entity_list = []
    for entity in document['entities']:
        if entity is None:
            break
        elif(entity['offsets'][0][0] > s_offsets[0][1]): 
            break
        elif (entity['offsets'][0][0] >=  s_offsets[0][0] and entity['offsets'][0][1] <= s_offsets[0][1]):
            entity_list.append(entity)
    return entity_list

tp = 0
e_count = 0
t_count = 0

def compare_findings(predi_entities, truth_entities, sentence):
    sentence_delta = sentence['offsets'][0][0]
    global tp, e_count, t_count
    for e in list(predi_entities.spans['snomed']):
        e_count+=1
        for t in truth_entities:
            if e.label_ == t['type'] and e.start_char == t['offsets'][0][0]-sentence_delta and e.end_char == t['offsets'][0][1]-sentence_delta:
                tp+=1
                break
    t_count += len(truth_entities)


json_path = "/Users/leon.sarodnik/Documents/GitHub/ggponc_annotation/grassco_project_2023-01-05_0021/grassco_converted/json/fine/long/test.json"

with open(json_path) as json_file:
    data = json.load(json_file)

for document in data:
    print("Current document_id: "+document['document_id'])
    for sentence in document['passages']:
        nlp_findings = nlp(sentence['text'], disable=["tok2vec", "tagger", "attribute_ruler","lemmatizer"]) # use pipe, disable_components in pipe (standard NER tagger etc.) - "parser" is helpful (10 matches less when not used)
        manual_findings = find_annotated_entities(sentence, document)
        compare_findings(nlp_findings, manual_findings, sentence)


In [ ]:
fn = t_count-tp
fp = e_count-tp

print(f'Actual Entities: {t_count}')
print(f'Predicted Entities: {e_count}')

print(f'True Positives: {tp}')
print(f'False Positives: {fp}')
print(f'False Negatives: {fn}')

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1_score:.2f}')

#result for fine/long

#Actual Entities: 6194
#Predicted Entities: 5171
#True Positives: 2584
#False Positives: 2587
#False Negatives: 3610
#Precision: 0.50
#Recall: 0.42
#F1 Score: 0.45

#result for fine/short
#Actual Entities: 7201
#Predicted Entities: 5171
#True Positives: 1737
#False Positives: 3434
#False Negatives: 5464
#Precision: 0.34
#Recall: 0.24
#F1 Score: 0.28

In [ ]:
for s in sorted(list(doc.spans['snomed']), key=lambda s: s.start):
    print(s, s.label_)